In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv('credit_approval_anonymized.csv') 

df.replace('', np.nan, inplace=True)

df.dropna(inplace=True)
print(df.columns)
print(df.dtypes)

Index(['A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'A9', 'A10', 'A11',
       'A12', 'A13', 'A14', 'A15', 'A16'],
      dtype='object')
A1      object
A2     float64
A3     float64
A4      object
A5      object
A6      object
A7      object
A8     float64
A9      object
A10     object
A11      int64
A12     object
A13     object
A14    float64
A15      int64
A16     object
dtype: object
